<img src="common/fun_long_logo-01.png">

### **ThermoFun** is a universal open-source client that delivers thermodynamic properties of substances and reactions at the temperature and pressure of interest.

# Export logK values in [soltherm](https://pages.uoregon.edu/palandri/data/soltherm.xpt) PT grid format 
Use the ```ThermoEnginge``` class in a script to calculate logK values at given TP, store them in a list and finally, use the ```ThermoBatch``` class with the list of stored results to output them in a CSV file.

### Import ThermoFun python package 

In [ ]:
import thermofun.PyThermoFun as fun

### Import a thermodynamic database from a json file
ThermoFun is initialized with a ThermoDataSet. A ThermoDataSet is a collection of records of substances and/or reactions also known in the (geo)chemical community as a thermodynamic database, e.g. SUPCRT98, PSI-Nagra, CEMDATA18. In this example we use a provided database file ```aq17-gem-lma-thermofun.json.json```. This database contains both substance and reaction records. The reactions for all secondary substances were generated in ThermoMach reactions generator module based on the master species used in soltherm.

In [ ]:
database = fun.Database("databases/aq17-gem-lma-thermofun.json")

Initialize the T and P list as in soltherm, and an empty results list.

In [ ]:
Ts =  [0.01,  25,  50,  75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350,
       400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600];
Ps =  [1.000, 1.0133, 2.321, 4.758, 8.919, 15.537, 25.479, 39.737, 59.432, 85.839,120.458, 165.212, 
       200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000];
results = []

Initialize a ```ThermoEngine``` object (used for calculating the properties of substances/reactions at desired T and P). The solvent is needed internally for the T an P extrapolations.

In [ ]:
engine = fun.ThermoEngine(database)
engine.setSolventSymbol('H2O@')

Script for looping through reaction records, Ps and Ts, but also to set 99999.9999 for values outside the validity range (as in soltherm). To call the script for an individual reaction the loop through ```mapReactions``` has to be disabled and ```r``` replaced with a reaction symbol.

In [ ]:
PressuresAtSat = {}
for r in database.mapReactions().keys():
    for pbar in Ps:
        for tC in Ts:
            pPa = pbar*1e5; # to Pa
            tK = tC+273.15; # to K
            tpr = fun.ThermoPropertiesReaction()
            tpr.log_equilibrium_constant.val = 99999.9989
            prop_solvent = fun.PropertiesSolvent()
        
            try:
                if not (tC>350. and pbar <200.): 
                    prop_solvent = engine.propertiesSolvent(tK,pPa,"H2O@")
            
                if not (tC>350.):
                    if not tC in PressuresAtSat.keys():
                        PressuresAtSat[tC]= engine.propertiesSolvent(tK,0,"H2O@").pressure.val
                    if (pPa < PressuresAtSat[tC]): # if in vapor region
                        prop_solvent.density.val = 0.0
            
                if (prop_solvent.density.val > 350.0):
                    tpr = engine.thermoPropertiesReactionFromReactants(tK,pPa, r)
        
            except (RuntimeError):
                tpr.log_equilibrium_constant.val = 99999.9989
              
            results.append(tpr)

Output results using ```ThermoBatch```. ThermoBatch has special output functionality (e.g. write CSV files) that is not accessible in the ```ThermoEngine```. when we don't do batch calculations directly using ThermoBatch, we can instead provide a list of results that were calculated before. 

In [ ]:
batch = fun.ThermoBatch(engine)

Set calculation and output preferences.
For a list of properties set desired units. Default units for temperature and pressure are K and Pa. <br>
For a list of properties set desired significant digits after the (.) to be written in the result file.

In [ ]:
batch.setPropertiesUnits(["temperature", "pressure"],["degC","bar"])
batch.setPropertiesDigits(["logKr", "pressure"], [3, 3])
op = fun.BatchPreferences()
op.loopTemperatureThenPressure = False # has to be consistent with the order of calculations 
batch.setBatchPreferences(op)

Write the results to a CSV file. 

In [ ]:
batch.thermoPropertiesReaction(Ts, Ps, list(database.mapReactions().keys()), ["logKr"], results).toCSVPropertyGrid("test_grid.csv")

Import functions for creating csv download link and data plotting

In [ ]:
from common.functions import create_csv_download_link, plot_substances_properties_vs_temperature

### Download the results file

In [ ]:
create_csv_download_link("logKr_test_grid.csv")